# MiniGrid Environment

Try out the environment by running the following command:


```bash
python -m minigrid.manual_control
```

Later we can benchmark against torch-rl 


In [23]:
env.observation_space

Box(0, 255, (7, 7, 3), uint8)

In [22]:
import gymnasium as gym
from minigrid.wrappers import RGBImgPartialObsWrapper, ImgObsWrapper

env = gym.make('MiniGrid-Empty-8x8-v0')
# env = RGBImgPartialObsWrapper(env) # Get pixel observations
env = ImgObsWrapper(env) # Get rid of the 'mission' field
obs, _ = env.reset() # This now produces an RGB tensor only
obs

array([[[2, 5, 0],
        [2, 5, 0],
        [2, 5, 0],
        [2, 5, 0],
        [2, 5, 0],
        [2, 5, 0],
        [2, 5, 0]],

       [[2, 5, 0],
        [2, 5, 0],
        [2, 5, 0],
        [2, 5, 0],
        [2, 5, 0],
        [2, 5, 0],
        [2, 5, 0]],

       [[2, 5, 0],
        [2, 5, 0],
        [2, 5, 0],
        [2, 5, 0],
        [2, 5, 0],
        [2, 5, 0],
        [2, 5, 0]],

       [[2, 5, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]],

       [[2, 5, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]],

       [[2, 5, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]],

       [[2, 5, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]]], dtype=uint8)

In [12]:
import torch as t 
import plotly.express as px
obs = t.tensor(obs)
obs.shape
px.imshow(obs)

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/Users/josephbloom/miniforge3/envs/decision_transformer_interpretability/lib/python3.9/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [13]:
env = gym.make('MiniGrid-Empty-8x8-v0')
env = RGBImgPartialObsWrapper(env) # Get pixel observations
env = ImgObsWrapper(env) # Get rid of the 'mission' field
obs, _ = env.reset() # This now produces an RGB tensor only

# take several actions, store the observations, actions, returns and timesteps
all_obs = []
all_actions = []
all_returns = []
all_timesteps = []


for i in range(10):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    all_obs.append(obs)
    all_actions.append(action)
    all_returns.append(reward)
    all_timesteps.append(i)

# convert to tensors.unsqueeze(0)
all_obs = t.tensor(all_obs)
all_actions = t.tensor(all_actions).reshape(-1, 1)
all_returns = t.tensor(all_returns)
all_returns = t.randn((10, 1))
all_returns_to_go = all_returns.flip(0).cumsum(0).flip(0).reshape(-1, 1)
all_timesteps = t.tensor(all_timesteps).reshape(-1, 1)

/var/folders/qd/04zj2h4n2pvc87x_7w1z97_r0000gn/T/ipykernel_93543/941578228.py:22: UserWarning:

Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:233.)



In [124]:
print(all_returns.shape)

torch.Size([10, 1])


In [125]:
print(all_returns_to_go.shape)

torch.Size([10, 1])


# Getting a basic architecture


In [126]:
# for the grid world environment we will a small CNN to extract features from the image
# we will use the same CNN as in the original paper

obs, _, _, _, _ = env.step(2)
obs = t.tensor(obs)

import torch as t
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange

# to do: make this a custom class with hooks from transformer lense
# to do: work out how to feature visualize this

class StateEncoder(nn.Module):
    def __init__(self, n_embed):
        super(StateEncoder, self).__init__()
        self.n_embed = n_embed
        # input has shape 56 x 56 x 3
        # output has shape 1 x 1 x 512
        self.conv1 = nn.Conv2d(3, 32, 8, stride=4, padding=0) # 56 -> 13
        self.conv2 = nn.Conv2d(32, 64, 4, stride=2, padding=0) # 13 -> 5
        self.conv3 = nn.Conv2d(64, 64, 3, stride=1, padding=0) # 5 -> 3
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(576, n_embed)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.flatten(x)
        x = self.fc(x)
        x = F.relu(x)
        return x

# we will use the same CNN as in the original paper
cnn = StateEncoder(64).to("cpu")
x = obs.unsqueeze(0).to(t.float32)
x = rearrange(x, 'b h w c-> b c h w')
cnn(x)

tensor([[ 0.0000,  0.0000,  4.6144,  1.2588,  0.1821,  0.0000,  0.0000,  0.0000,
          0.0000, 10.2853,  5.1550,  0.0000,  0.0000,  0.0000,  0.0777,  0.8252,
          1.6667,  3.1823,  1.6241,  0.0000,  2.2314,  1.1757,  1.5849,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.0670,
          2.2828,  0.0000,  0.0000,  4.6453,  0.0000,  7.2832,  0.0000,  1.5017,
          0.0000,  1.6794,  0.0000,  0.0000,  0.0000,  0.0000,  6.0537,  0.0000,
          9.3869,  3.8999,  0.0000,  1.0585,  0.0000,  0.0000,  0.0000,  4.4552,
          0.0000,  0.0000,  3.5174,  0.0000,  0.0000,  2.0047,  0.5330,  0.0000]],
       grad_fn=<ReluBackward0>)

For reference: https://github.com/kzl/decision-transformer/blob/master/atari/mingpt/model_atari.py

In [127]:

    # def forward(self, R: t.tensor, s: t.tensor, a: t.tensor, t: t.tensor):
    #     '''
    #     R: return
    #     s: state
    #     a: action
    #     t: timestep
    #     '''
    #     s = s.to(torch.float32)
    #     s = rearrange(s, 'b h w c-> b c h w')

    #     pos_emb = self.pos_embedding(t)
    #     state_emb = self.state_embedding(s) + pos_emb
    #     action_emb = self.action_embeddings(a) + pos_emb
    #     ret_emb = self.ret_emb(R) + pos_emb

    #     input_embeds = torch.stack([state_emb, action_emb, ret_emb], dim=1)
    #     print(input_embeds.shape)
    #     print(input_embeds.dtype)
    #     input_embeds = rearrange(input_embeds, 'batch sar block_size d_model -> batch sar block_size d_model')
        
    #     x = self.transformer(input_embeds)
        
    #     return x


Notes:
- This turned out to be really complicated. 
- Specifically:
    - it seems like the model has very different formulations during reward_conditioned vs naive. And also changes size if targets are used.
    - This includes things like the token embeddings for actions not going through in naive mode, but going through in reward_conditioned mode.
        - This is fine because a model is only ever of one type
    - However, the model isn't using padding of any kind? I should look for evidence of this. 

In [128]:
all_timesteps.unsqueeze(1).shape

torch.Size([10, 1, 1])

In [129]:
all_timesteps.repeat(3,1, 1).shape

torch.Size([3, 10, 1])

In [130]:
print(all_actions.repeat(3,1, 1).shape)
print(all_returns_to_go.repeat(3,1, 1).shape)
print(all_timesteps.repeat(3,1, 1).shape)
print(all_obs.repeat(3,1, 1, 1, 1).shape)

torch.Size([3, 10, 1])
torch.Size([3, 10, 1])
torch.Size([3, 10, 1])
torch.Size([3, 10, 56, 56, 3])


In [131]:
rearrange(t.tensor([[1,2],[4,5],[7,8]]), 'b (n d) -> b n d', n=1, d=2)

tensor([[[1, 2]],

        [[4, 5]],

        [[7, 8]]])

In [132]:

# reference code:

# all_global_pos_emb = torch.repeat_interleave(self.global_pos_emb, batch_size, dim=0) # batch_size, traj_length, n_embd
# position_embeddings = torch.gather(all_global_pos_emb, 1, torch.repeat_interleave(timesteps, self.config.n_embd, dim=-1)) + self.pos_emb[:, :token_embeddings.shape[1], :]
# x = self.drop(token_embeddings + position_embeddings)

In [133]:
import torch as t 
import gymnasium as gym
from minigrid.wrappers import RGBImgPartialObsWrapper, ImgObsWrapper

env = gym.make('MiniGrid-Empty-8x8-v0')
env = RGBImgPartialObsWrapper(env) # Get pixel observations
env = ImgObsWrapper(env) # Get rid of the 'mission' field
obs, _ = env.reset() # This now produces an RGB tensor only

# take several actions, store the observations, actions, returns and timesteps
all_obs = []
all_actions = []
all_returns = []
all_timesteps = []


for i in range(10):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    all_obs.append(obs)
    all_actions.append(action)
    all_returns.append(reward)
    all_timesteps.append(i)

# convert to tensors.unsqueeze(0)
all_obs = t.tensor(all_obs).to(t.float32).unsqueeze(0)
all_actions = t.tensor(all_actions).reshape(-1, 1).unsqueeze(0)
all_returns = t.randn((10, 1))
all_returns_to_go = all_returns.flip(0).cumsum(0).flip(0).reshape(-1, 1).unsqueeze(0)
all_timesteps = t.tensor(all_timesteps).reshape(-1, 1).unsqueeze(0)


# Train a decision transformer on minigrid

To do this we will need:
- example trajectories (offline learning)
- a training loop

To make the sample trajectories we need:
- an agent which can traverse those maps (better if it's something good like PPO) (Agent class)
- a way to sample trajectories from that agent (preferably parallelizable)

In [134]:
import numpy as np 
ActType = Union[int, np.ndarray]

class Agent:
    '''Base class for agents in a multi-armed bandit environment (you do not need to add any implementation here)'''

    rng: np.random.Generator

    def __init__(self, num_arms: int, seed: int):
        self.num_arms = num_arms
        self.reset(seed)

    def get_action(self) -> ActType:
        raise NotImplementedError()

    def observe(self, action: ActType, reward: float, info: dict) -> None:
        pass

    def reset(self, seed: int) -> None:
        self.rng = np.random.default_rng(seed)

class RandomAgent(Agent):
    def __init__(self, env):
        self.env = env
    def get_action(self):
        return self.env.action_space.sample()

def run_episode(env: gym.Env, agent: Agent, seed: int):
    rewards = []
    actions = []
    states = []
    env.reset(seed=seed)
    agent.reset(seed=seed)
    done = False
    while not done:
        arm = agent.get_action()
        actions.append(arm)
        (obs, reward, done, truncated, info) = env.step(arm)
        agent.observe(arm, reward, info)
        states.append(obs)
        rewards.append(reward)
    rewards = np.array(rewards, dtype=float)
    actions = np.array(actions, dtype=int)

    return rewards, np.array(states), actions

env = gym.make('MiniGrid-Empty-5x5-v0')
env = RGBImgPartialObsWrapper(env) # Get pixel observations
env = ImgObsWrapper(env) # Get rid of the 'mission' field
agent = RandomAgent(env)

reward_trajs = []
states_trajs = []
actions_trajs = []
for event in range(1000):
    reward_traj, states_traj, actions_traj = run_episode(env, agent, seed=i)
    reward_trajs.append(reward_traj)
    states_trajs.append(states_traj)
    actions_trajs.append(actions_traj)

# gym.vector.SyncVectorEnv(
#     env_fns=[lambda: gym.make('MiniGrid-Empty-5x5-v0') for _ in range(10)],
# )

reward_trajs = np.array(reward_trajs)
states_trajs = np.array(states_trajs)
actions_trajs = np.array(actions_trajs)


/var/folders/qd/04zj2h4n2pvc87x_7w1z97_r0000gn/T/ipykernel_91131/4066994141.py:65: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/var/folders/qd/04zj2h4n2pvc87x_7w1z97_r0000gn/T/ipykernel_91131/4066994141.py:66: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/var/folders/qd/04zj2h4n2pvc87x_7w1z97_r0000gn/T/ipykernel_91131/4066994141.py:67: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you mus

In [135]:
print(reward_trajs.shape)
print(states_trajs.shape)
print(actions_trajs.shape)

(1000,)
(1000,)
(1000,)


In [136]:
lengths = np.array([len(reward_traj) for reward_traj in reward_trajs])
print(lengths.shape)

(1000,)


In [137]:
import plotly.express as px 
px.histogram(lengths)

In [138]:
# let's try an example with a single trajectory
reward_traj = reward_trajs[0]
states_traj = states_trajs[0]
actions_traj = actions_trajs[0]

reward_traj

rtg = np.flip(reward_traj).cumsum(0)



decision_transformer = DecisionTransformer(env, max_game_length= 10**4)

logits, _ = decision_transformer(
    states = t.tensor(states_traj).to(t.float32).unsqueeze(0),
    actions = t.tensor(actions_traj).unsqueeze(0).unsqueeze(-1),
    rtgs = t.tensor(rtg).unsqueeze(0).unsqueeze(-1),
    timesteps = t.tensor(np.arange(len(reward_traj))).unsqueeze(0).unsqueeze(-1)
)

ValueError: Trajectory length is greater than the maximum sequence length for this model

In [ ]:
t.tensor(reward_traj).unsqueeze(-1).unsqueeze(0).shape

torch.Size([1, 180, 1])

In [ ]:
t.tensor(np.arange(len(reward_traj))).unsqueeze(0).unsqueeze(-1).shape

torch.Size([1, 180, 1])

In [ ]:
gym.__version__

'0.27.0'

# See if I can load a replay buffer from D4RL


In [3]:
import d4rl
import gym 
import minigrid
from minigrid.wrappers import RGBImgPartialObsWrapper, ImgObsWrapper
from warnings import simplefilter
simplefilter(action='ignore', category=DeprecationWarning)
env = gym.make('maze2d-eval-medium-v1')
# _ = env.reset() # This now produces an RGB tensor only
env.get_dataset()

load datafile: 100%|██████████| 8/8 [00:00<00:00,  8.50it/s]


{'actions': array([[ 0.12521252, -0.6819198 ],
        [-1.        , -1.        ],
        [-0.24480188, -0.59214544],
        ...,
        [-0.86986226, -1.        ],
        [-1.        , -0.36177555],
        [-1.        , -0.15718542]], dtype=float32),
 'infos/goal': array([[5.995181 , 0.912286 ],
        [5.995181 , 0.912286 ],
        [5.995181 , 0.912286 ],
        ...,
        [0.9864979, 6.004362 ],
        [0.9864979, 6.004362 ],
        [0.9864979, 6.004362 ]], dtype=float32),
 'infos/qpos': array([[4.944978 , 4.084466 ],
        [4.945365 , 4.083295 ],
        [4.943369 , 4.0797443],
        ...,
        [3.6793704, 2.89519  ],
        [3.6387599, 2.9047458],
        [3.5958643, 2.9134173]], dtype=float32),
 'infos/qvel': array([[ 0.0088652 ,  0.04536624],
        [ 0.03866518, -0.11715045],
        [-0.19959074, -0.35503528],
        ...,
        [-3.8630888 ,  1.19659   ],
        [-4.061058  ,  0.95557624],
        [-4.28955   ,  0.86713856]], dtype=float32),
 'observati

In [4]:
from gym import envs
import d4rl.gym_minigrid
all_envs = envs.registry.all()
env_ids = [env_spec.id for env_spec in all_envs]
print(sorted(env_ids))

['Acrobot-v1', 'Ant-v2', 'Ant-v3', 'AntBulletEnv-v0', 'BipedalWalker-v3', 'BipedalWalkerHardcore-v3', 'Blackjack-v1', 'CarRacing-v1', 'CartPole-v0', 'CartPole-v1', 'CartPoleBulletEnv-v1', 'CartPoleContinuousBulletEnv-v0', 'CliffWalking-v0', 'FrozenLake-v1', 'FrozenLake8x8-v1', 'HalfCheetah-v2', 'HalfCheetah-v3', 'HalfCheetahBulletEnv-v0', 'Hopper-v2', 'Hopper-v3', 'HopperBulletEnv-v0', 'Humanoid-v2', 'Humanoid-v3', 'HumanoidBulletEnv-v0', 'HumanoidDeepMimicBackflipBulletEnv-v1', 'HumanoidDeepMimicWalkBulletEnv-v1', 'HumanoidFlagrunBulletEnv-v0', 'HumanoidFlagrunHarderBulletEnv-v0', 'HumanoidStandup-v2', 'InvertedDoublePendulum-v2', 'InvertedDoublePendulumBulletEnv-v0', 'InvertedPendulum-v2', 'InvertedPendulumBulletEnv-v0', 'InvertedPendulumSwingupBulletEnv-v0', 'KukaBulletEnv-v0', 'KukaCamBulletEnv-v0', 'KukaDiverseObjectGrasping-v0', 'LunarLander-v2', 'LunarLanderContinuous-v2', 'MinitaurAlternatingLegsEnv-v0', 'MinitaurBallGymEnv-v0', 'MinitaurBulletDuckEnv-v0', 'MinitaurBulletEnv-v0

In [5]:
import gym 
env = gym.make('minigrid-fourrooms-v0')
# _ = env.reset() # This now produces an RGB tensor only
dataset = env.get_dataset()

load datafile: 100%|██████████| 8/8 [00:00<00:00, 29.12it/s]


In [6]:
dataset.keys()

dict_keys(['actions', 'infos/goal', 'infos/orientation', 'infos/pos', 'observations', 'rewards', 'terminals', 'timeouts'])

In [30]:
# count all the unique values of rewards
np.unique(dataset['rewards'])

array([0.], dtype=float32)

In [9]:
dataset["observations"][0]

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [2., 5., 0.],
        [2., 5., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [2., 5., 0.],
        [1., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [2., 5., 0.],
        [1., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [2., 5., 0.],
        [1., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]